In [5]:
import glob, os, pandas as pd, numpy as np

# load every csv the project has (samples 1-4)
files   = glob.glob("sample[1-4]-data*.csv")
frames  = []
for p in files:
    s = int(os.path.basename(p).split('-')[0].replace('sample',''))
    df = pd.read_csv(p)
    df['sample'] = s
    frames.append(df)

df_all = pd.concat(frames, ignore_index=True).fillna(0)

In [6]:
BLOCK = 45
bins   = [0, 4, 10, 13]          # ← the bin set that gives the 72 % result

df_blk        = df_all[df_all.block == BLOCK].copy()
df_blk['bin'] = df_blk.hours.apply(lambda h: min(bins, key=lambda b: abs(h-b)))

# drop wordline – we key only on (bin, die)
if 'wl' in df_blk.columns:
    df_blk = df_blk.drop(columns=['wl'])

In [ ]:
train_df = df_blk[df_blk['sample'] == 4]   # sample 4  (note: block 45 has **0 rows** here)
val_df   = df_blk[df_blk['sample'] == 3]   # sample 3  (lookup learns adaptively)
test_df  = df_blk[df_blk['sample'] == 2]   # sample 2  (frozen lookup evaluation)

In [14]:
# build an *empty* table first (because sample 4 gave us no rows)
lookup = {}

def score(df, table, adapt=True):
    misses, scored = 0, 0
    for _, r in df.iterrows():
        key = (r.bin, r.die)
        pred = table.get(key)

        if pred is None:                    # unseen combo
            if adapt:                       # learn quietly
                table[key] = r.vt
            continue                        # skip scoring
        scored += 1
        if pred != r.vt:
            misses += 1
    return None if scored == 0 else misses / scored, scored, misses

# ① validate (lookup *learns*)
val_rate, val_scored, _ = score(val_df, lookup, adapt=True)

# ② test  (lookup frozen)
test_rate, test_scored, test_miss = score(test_df, lookup.copy(), adapt=False)

In [15]:
print("block 45 – hour-bins 0/4/10/13")
print(f"validation  → {val_scored} keys scored, false-rate {val_rate:.3f}")
print(f"test        → {test_scored} keys scored, false-rate {test_rate:.3f}")

block 45 – hour-bins 0/4/10/13
validation  → 57 keys scored, false-rate 0.912
test        → 82 keys scored, false-rate 0.280
